## Thanks:
Mark Peng, for precompiling AlexeyAB's Darknet to work on the Kaggle enviornment: [Notebook](https://www.kaggle.com/markpeng/darknet-gpu-on-kaggle/?)

Alex Shonenkov for his notebook detailing WBF over TTA for Efficientdet: [Notebook](https://www.kaggle.com/shonenkov/wbf-over-tta-single-model-efficientdet)

Tianxiaomo for a great library to convert and infer Darknet models in Pytorch, and for accepting my pull requests: [Github Repo](https://github.com/Tianxiaomo/pytorch-YOLOv4)

## Info
This notebook is a complete inference for a single YOLOv4 model trained using Darknet. Inference is done with TTAx4 at 1024x1024 resolution, while training is done with randaug and 704x704 resolution.

Please see [Version 19](https://www.kaggle.com/stanleyjzheng/apache2-yolov4-pseudolabelling-oof?scriptVersionId=40172709) for a notebook that would have gotten 38th place in this competition.

In [ ]:
!cp -r ../input/pytorchyolov4/* .
#!cp ../input/yolov4weights/obj.names * 
#!cp ../input/yolov4weights/trainsubmit.data * 
#!cp ../input/yolov4weights/trainsubmit.cfg * 
!mkdir /kaggle/working/backup

In [ ]:
!cp -r "../input/darknet/darknet" .
!cp -r "../input/darknet/libdarknet.so" .
!cp -r "../input/darknet/darknet.py" .
!chmod a+x darknet

In [ ]:
import numpy as np
import pandas as pd
import os
from tqdm.auto import tqdm
import shutil as sh
import sys
import glob
import cv2

sys.path.insert(0, "../input/weightedboxesfusion/")
from ensemble_boxes import *
from tool.utils import *
from tool.torch_utils import *
from tool.darknet2pytorch import Darknet

NMS_IOU_THR = 0.6
NMS_CONF_THR = 0.5

# WBF
best_iou_thr = 0.42
best_skip_box_thr = 0.41

# Box conf threshold
best_final_score = 0
best_score_threshold = 0

cfgfile = '../input/yolov4weights/submit.cfg'
weightfile = '../input/yolov4weights/submit.weights'

WEIGHTS = Darknet(cfgfile)
WEIGHTS.load_weights(weightfile)
import torch
WEIGHTS.cuda()

is_TEST = len(os.listdir('../input/global-wheat-detection/test/'))>11

is_AUG = True
is_ROT = True

VALIDATE = True

PSEUDO = True

# For OOF evaluation
marking = pd.read_csv('../input/global-wheat-detection/train.csv')

bboxs = np.stack(marking['bbox'].apply(lambda x: np.fromstring(x[1:-1], sep=',')))
for i, column in enumerate(['x', 'y', 'w', 'h']):
    marking[column] = bboxs[:,i]
marking.drop(columns=['bbox'], inplace=True)

In [ ]:
import ast
import random
def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return [x, y, w, h]

def convert_to_yolo_label(coco_format_box, w = 1024, h = 1024):
    bbox = ast.literal_eval(coco_format_box)
    xmin = bbox[0]
    xmax = bbox[0] + bbox[2]
    ymin = bbox[1]
    ymax = bbox[1] + bbox[3]
    b = (float(xmin), float(xmax), float(ymin), float(ymax))
    yolo_box = convert((w, h), b)
    if np.max(yolo_box) > 1 or np.min(yolo_box) < 0:
        print("BOX HAS AN ISSUE")
    return yolo_box

def convertTrainLabel():
    df = pd.read_csv('../input/global-wheat-detection/train.csv')
    from tqdm.auto import tqdm
    import shutil as sh
    index = list(set(df.image_id))
    df['yolo_box'] = df.bbox.apply(convert_to_yolo_label)
    unique_img_ids = df.image_id.unique()
    for fold in [0]:
        val_index = index[len(index)*fold//5:len(index)*(fold+1)//5]
        source = 'train'
        for img_id in unique_img_ids:
            filt_df = df.query("image_id == @img_id")
            path2save = 'val2017/' if img_id in val_index else 'train2017/'
            os.makedirs('convertor/fold{}/{}'.format(fold,path2save), exist_ok=True)
            folder_location = 'convertor/fold0/'+path2save
            all_boxes = filt_df.yolo_box.values
            file_name = "{}/{}.txt".format(folder_location,img_id)
            s = "0 %s %s %s %s \n"
            with open(file_name, 'a') as file:
                for i in all_boxes:
                    new_line = (s % tuple(i))
                    file.write(new_line)
            sh.copy("../input/global-wheat-detection/{}/{}.jpg".format(source,img_id),'convertor/fold{}/{}/{}.jpg'.format(fold,path2save,img_id))
        print("finished positive images")
        all_imgs = glob.glob("../input/global-wheat-detection/train/*.jpg")
        all_imgs = [i.split("/")[-1].replace(".jpg", "") for i in all_imgs]
        positive_imgs = df.image_id.unique()
        negative_images = set(all_imgs) - set(positive_imgs)
        folder_location = 'convertor/fold0/train2017/'
        for i in list(negative_images):
            file_name = folder_location+"{}.txt".format(i)
            sh.copy("../input/global-wheat-detection/train/{}.jpg".format(i),'convertor/fold0/train2017/{}.jpg'.format(i))
            with open(file_name, 'w') as fp: 
                pass
        print('finished negative images')

def run_wbf(boxes, scores, image_size=1024, iou_thr=0.5, skip_box_thr=0.7, weights=None):
    labels = [np.zeros(score.shape[0]) for score in scores]
    boxes = [box/(image_size) for box in boxes]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size)
    return boxes, scores, labels

def TTAImage(image, index):
    image1 = image.copy()
    if index==0: 
        rotated_image = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image
    elif index==1:
        rotated_image2 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image2 = cv2.rotate(rotated_image2, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image2
    elif index==2:
        rotated_image3 = cv2.rotate(image1, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        rotated_image3 = cv2.rotate(rotated_image3, cv2.ROTATE_90_CLOCKWISE)
        return rotated_image3
    elif index == 3:
        return image1

def rotBoxes90(boxes, im_w, im_h):
    ret_boxes =[]
    for box in boxes:
        x1, y1, x2, y2 = box
        x1, y1, x2, y2 = x1-im_w//2, im_h//2 - y1, x2-im_w//2, im_h//2 - y2
        x1, y1, x2, y2 = y1, -x1, y2, -x2
        x1, y1, x2, y2 = int(x1+im_w//2), int(im_h//2 - y1), int(x2+im_w//2), int(im_h//2 - y2)
        x1a, y1a, x2a, y2a = min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)
        ret_boxes.append([x1a, y1a, x2a, y2a])
    return np.array(ret_boxes)

def process_det(index, det, imgsize=1024, score_threshold=0.35):
    try :
        scores = det[index][:, 5].copy()
        det = det[index][:, :4].copy()
        bboxes = np.zeros((det.shape))
        bboxes[:, 0] = (det[:, 0] * imgsize).astype(int)
        bboxes[:, 1] = (det[:, 1] * imgsize).astype(int)
        bboxes[:, 2] = (det[:, 2] * imgsize).astype(int)
        bboxes[:, 3] = (det[:, 3] * imgsize).astype(int)
        bboxes = (bboxes).clip(min = 0, max = imgsize).astype(int)
        indexes = np.where(scores>score_threshold)
        bboxes = bboxes[indexes]
        scores = scores[indexes]
        return bboxes, scores
    except IndexError:
        print(det)


def detect1Image(img, img0, model, device, aug):
    pred = do_detect(model, img, NMS_CONF_THR, NMS_IOU_THR, use_cuda=1)

    prednp = np.array(pred)
    if prednp is None or prednp.size==0:
        return np.array([]), np.array([])
    else:
        boxes, scores = process_det(0, prednp)
        return np.array(boxes), np.array(scores)

In [ ]:
import pandas as pd
import numpy as np
import numba
import re
import cv2
import matplotlib.pyplot as plt

from numba import jit
from typing import List, Union, Tuple


@jit(nopython=True)
def calculate_iou(gt, pr, form='pascal_voc') -> float:
    """Calculates the Intersection over Union.

    Args:
        gt: (np.ndarray[Union[int, float]]) coordinates of the ground-truth box
        pr: (np.ndarray[Union[int, float]]) coordinates of the prdected box
        form: (str) gt/pred coordinates format
            - pascal_voc: [xmin, ymin, xmax, ymax]
            - coco: [xmin, ymin, w, h]
    Returns:
        (float) Intersection over union (0.0 <= iou <= 1.0)
    """
    if form == 'coco':
        gt = gt.copy()
        pr = pr.copy()

        gt[2] = gt[0] + gt[2]
        gt[3] = gt[1] + gt[3]
        pr[2] = pr[0] + pr[2]
        pr[3] = pr[1] + pr[3]

    # Calculate overlap area
    dx = min(gt[2], pr[2]) - max(gt[0], pr[0]) + 1
    
    if dx < 0:
        return 0.0
    
    dy = min(gt[3], pr[3]) - max(gt[1], pr[1]) + 1

    if dy < 0:
        return 0.0

    overlap_area = dx * dy
    
    # Calculate union area
    union_area = (
            (gt[2] - gt[0] + 1) * (gt[3] - gt[1] + 1) +
            (pr[2] - pr[0] + 1) * (pr[3] - pr[1] + 1) -
            overlap_area
    )

    return overlap_area / union_area


@jit(nopython=True)
def find_best_match(gts, pred, pred_idx, threshold = 0.5, form = 'pascal_voc', ious=None) -> int:
    """Returns the index of the 'best match' between the
    ground-truth boxes and the prediction. The 'best match'
    is the highest IoU. (0.0 IoUs are ignored).

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        pred: (List[Union[int, float]]) Coordinates of the predicted box
        pred_idx: (int) Index of the current predicted box
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (int) Index of the best match GT box (-1 if no match above threshold)
    """
    best_match_iou = -np.inf
    best_match_idx = -1

    for gt_idx in range(len(gts)):
        
        if gts[gt_idx][0] < 0:
            # Already matched GT-box
            continue
        
        iou = -1 if ious is None else ious[gt_idx][pred_idx]

        if iou < 0:
            iou = calculate_iou(gts[gt_idx], pred, form=form)
            
            if ious is not None:
                ious[gt_idx][pred_idx] = iou

        if iou < threshold:
            continue

        if iou > best_match_iou:
            best_match_iou = iou
            best_match_idx = gt_idx

    return best_match_idx

@jit(nopython=True)
def calculate_precision(gts, preds, threshold = 0.5, form = 'coco', ious=None) -> float:
    """Calculates precision for GT - prediction pairs at one threshold.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        threshold: (float) Threshold
        form: (str) Format of the coordinates
        ious: (np.ndarray) len(gts) x len(preds) matrix for storing calculated ious.

    Return:
        (float) Precision
    """
    n = len(preds)
    tp = 0
    fp = 0
    
    # for pred_idx, pred in enumerate(preds_sorted):
    for pred_idx in range(n):

        best_match_gt_idx = find_best_match(gts, preds[pred_idx], pred_idx,
                                            threshold=threshold, form=form, ious=ious)

        if best_match_gt_idx >= 0:
            # True positive: The predicted box matches a gt box with an IoU above the threshold.
            tp += 1
            # Remove the matched GT box
            gts[best_match_gt_idx] = -1

        else:
            # No match
            # False positive: indicates a predicted box had no associated gt box.
            fp += 1

    # False negative: indicates a gt box had no associated predicted box.
    fn = (gts.sum(axis=1) > 0).sum()

    return tp / (tp + fp + fn)


@jit(nopython=True)
def calculate_image_precision(gts, preds, thresholds = (0.5, ), form = 'coco') -> float:
    """Calculates image precision.

    Args:
        gts: (List[List[Union[int, float]]]) Coordinates of the available ground-truth boxes
        preds: (List[List[Union[int, float]]]) Coordinates of the predicted boxes,
               sorted by confidence value (descending)
        thresholds: (float) Different thresholds
        form: (str) Format of the coordinates

    Return:
        (float) Precision
    """
    n_threshold = len(thresholds)
    image_precision = 0.0
    
    ious = np.ones((len(gts), len(preds))) * -1
    # ious = None

    for threshold in thresholds:
        precision_at_threshold = calculate_precision(gts.copy(), preds, threshold=threshold,
                                                     form=form, ious=ious)
        image_precision += precision_at_threshold / n_threshold

    return image_precision

    
# Numba typed list!
iou_thresholds = numba.typed.List()

for x in [0.5, 0.55, 0.6, 0.65, 0.7, 0.75]:
    iou_thresholds.append(x)
    
def validate():
    source = 'convertor/fold0/val2017'
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Load model
    model = WEIGHTS
    model.to(device).eval()
    
    dataset = LoadImages(source, img_size=1024)

    results = []
    
    for path, img, img0, vid_cap in dataset:
            
        image_id = os.path.basename(path).split('.')[0]
        img = img.transpose(1,2,0) # [H, W, 3]
        
        enboxes = []
        enscores = []
        
        # only rot, no flip
        if is_ROT:    
            for i in range(4):
                img1 = TTAImage(img, i)
                boxes, scores = detect1Image(img1, img0, model, device, aug=False)
                for _ in range(3-i):
                    boxes = rotBoxes90(boxes, *img.shape[:2])            
                enboxes.append(boxes)
                enscores.append(scores) 
        
        # flip
        boxes, scores = detect1Image(img, img0, model, device, aug=is_AUG)
        enboxes.append(boxes)
        enscores.append(scores) 
            
        boxes, scores, labels = run_wbf(enboxes, enscores, image_size=1024, iou_thr=best_iou_thr, skip_box_thr=best_skip_box_thr)    
        boxes = boxes.astype(np.int32).clip(min=0, max=1024)

        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        boxes = boxes[scores >= 0.05].astype(np.int32)
        scores = scores[scores >= float(0.05)]
        
        records = marking[marking['image_id'] == image_id]
        gtboxes = records[['x', 'y', 'w', 'h']].values
        gtboxes = gtboxes.astype(np.int32).clip(min=0, max=1024)
        gtboxes[:, 2] = gtboxes[:, 0] + gtboxes[:, 2]
        gtboxes[:, 3] = gtboxes[:, 1] + gtboxes[:, 3]
        
            
        result = {
            'image_id': image_id,
            'pred_enboxes': enboxes, # xyhw
            'pred_enscores': enscores,
            'gt_boxes': gtboxes, # xyhw
        }

        results.append(result)
        
    return results

def calculate_final_score(all_predictions, iou_thr, skip_box_thr, score_threshold):
    final_scores = []
    for i in range(len(all_predictions)):
        gt_boxes = all_predictions[i]['gt_boxes'].copy()
        enboxes = all_predictions[i]['pred_enboxes'].copy()
        enscores = all_predictions[i]['pred_enscores'].copy()
        image_id = all_predictions[i]['image_id']
        
        pred_boxes, scores, labels = run_wbf(enboxes, enscores, image_size=1024, iou_thr=iou_thr, skip_box_thr=skip_box_thr)    
        pred_boxes = pred_boxes.astype(np.int32).clip(min=0, max=1024)

        indexes = np.where(scores>score_threshold)
        pred_boxes = pred_boxes[indexes]
        scores = scores[indexes]

        image_precision = calculate_image_precision(gt_boxes, pred_boxes,thresholds=iou_thresholds,form='pascal_voc')
        final_scores.append(image_precision)

    return np.mean(final_scores)

In [ ]:
!pip install scikit-optimize
from skopt import gp_minimize, forest_minimize
from skopt.utils import use_named_args
from skopt.plots import plot_objective, plot_evaluations, plot_convergence, plot_regret
from skopt.space import Categorical, Integer, Real
from pathlib import Path
from sys import exit

def log(text):
    print(text)

def optimize(space, all_predictions, n_calls=10):
    @use_named_args(space)
    def score(**params):
        log('-'*10)
        log(params)
        final_score = calculate_final_score(all_predictions, **params)
        log(f'final_score = {final_score}')
        log('-'*10)
        return -final_score

    return gp_minimize(func=score, dimensions=space, n_calls=n_calls) 

def letterbox(img, new_shape=(416, 416), color=(114, 114, 114), auto=True, scaleFill=False, scaleup=True):
    shape = img.shape[:2]  
    if isinstance(new_shape, int):
        new_shape = (new_shape, new_shape)

    r = min(new_shape[0] / shape[0], new_shape[1] / shape[1])
    if not scaleup:
        r = min(r, 1.0)

    ratio = r, r 
    new_unpad = int(round(shape[1] * r)), int(round(shape[0] * r))
    dw, dh = new_shape[1] - new_unpad[0], new_shape[0] - new_unpad[1]  
    if auto:
        dw, dh = np.mod(dw, 64), np.mod(dh, 64)
    elif scaleFill:
        dw, dh = 0.0, 0.0
        new_unpad = new_shape
        ratio = new_shape[0] / shape[1], new_shape[1] / shape[0]

    dw /= 2 
    dh /= 2

    if shape[::-1] != new_unpad:
        img = cv2.resize(img, new_unpad, interpolation=cv2.INTER_LINEAR)
    top, bottom = int(round(dh - 0.1)), int(round(dh + 0.1))
    left, right = int(round(dw - 0.1)), int(round(dw + 0.1))
    img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)  # add border
    return img, ratio, (dw, dh)

class LoadImages:  
    def __init__(self, path, img_size=416):
        path = str(Path(path))
        files = []
        if os.path.isdir(path):
            files = sorted(glob.glob(os.path.join(path, '*.*')))
        elif os.path.isfile(path):
            files = [path]

        img_formats = ['.bmp', '.jpg', '.jpeg', '.png', '.tif', '.tiff', '.dng']
        images = [x for x in files if os.path.splitext(x)[-1].lower() in img_formats]
        videos = []
        nI, nV = len(images), len(videos)

        self.img_size = img_size
        self.files = images + videos
        self.nF = nI + nV  # number of files
        self.video_flag = [False] * nI + [True] * nV
        self.mode = 'images'
        if any(videos):
            self.new_video(videos[0])  
        else:
            self.cap = None
        assert self.nF > 0, 'No images or found in %s. Supported formats are:\nimages: %s\nvideos: %s' % \
                            (path)

    def __iter__(self):
        self.count = 0
        return self

    def __next__(self):
        if self.count == self.nF:
            raise StopIteration
        path = self.files[self.count]
        self.count += 1
        img0 = cv2.imread(path)
        assert img0 is not None, 'Image Not Found ' + path
        print('image %g/%g %s: ' % (self.count, self.nF, path), end='')

        img = letterbox(img0, new_shape=self.img_size)[0]

        img = img[:, :, ::-1].transpose(2, 0, 1) 
        img = np.ascontiguousarray(img)

        return path, img, img0, self.cap

    def __len__(self):
        return self.nF

In [ ]:
def makePseudolabel():
    source = '../input/global-wheat-detection/test/'
    
    imagenames =  os.listdir(source)
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    model = WEIGHTS
    model.to(device).eval()
    
    dataset = LoadImages(source, img_size=1024)

    path2save = 'train2017/'
    
    if not os.path.exists('convertor/fold0/labels/'+path2save):
        os.makedirs('convertor/fold0/labels/'+path2save)
    if not os.path.exists('convertor/fold0/images/{}'.format(path2save)):
        os.makedirs('convertor/fold0/images/{}'.format(path2save))
    
    for path, img, img0, vid_cap in dataset:
        image_id = os.path.basename(path).split('.')[0]
        img = img.transpose(1,2,0)
        
        enboxes = []
        enscores = []
        
        if is_ROT:    
            for i in range(4):
                img1 = TTAImage(img, i)
                boxes, scores = detect1Image(img1, img0, model, device, aug=False)
                for _ in range(3-i):
                    boxes = rotBoxes90(boxes, *img.shape[:2])            
                enboxes.append(boxes)
                enscores.append(scores) 
        
        boxes, scores = detect1Image(img, img0, model, device, aug=is_AUG)
        enboxes.append(boxes)
        enscores.append(scores)
        boxes, scores, labels = run_wbf(enboxes, enscores, image_size=1024, iou_thr=best_iou_thr, skip_box_thr=best_skip_box_thr)
        boxes = boxes.astype(np.int32).clip(min=0, max=1024)

        indices = scores >= best_score_threshold
        boxes = boxes[indices]
        scores = scores[indices] #Right now, x1, y1, x2, y2

        boxes[:, 2] = boxes[:, 2] - boxes[:, 0] #convert to x1, y1, w, h
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1] 
 
        yolo_box = []
        for bbox in boxes:
            xmin = bbox[0]
            xmax = bbox[0] + bbox[2]
            ymin = bbox[1]
            ymax = bbox[1] + bbox[3]
            b = (float(xmin), float(xmax), float(ymin), float(ymax))
            yolo_box.append(convert((1024, 1024), b))
        
        folder_location = 'convertor/fold0/'+path2save
        file_name = "{}/{}.txt".format(folder_location,image_id) 
        s = "0 %s %s %s %s \n" 
        with open(file_name, 'w+') as file:
            for i in yolo_box:
                new_line = (s % tuple(i))
                file.write(new_line)

        sh.copy("../input/global-wheat-detection/test/{}.jpg".format(image_id),'convertor/fold0/{}/{}.jpg'.format(path2save,image_id))

def generate_train():
    image_files = []
    os.chdir(os.path.join("convertor", "fold0", "train2017"))
    for filename in os.listdir(os.getcwd()):
        if filename.endswith(".jpg"):
            image_files.append("/kaggle/working/convertor/fold0/train2017/" + filename)
    os.chdir("..")
    with open("train.txt", "w+") as outfile:
        for image in image_files:
            outfile.write(image)
            outfile.write("\n")
        outfile.close()
    os.chdir("..")
    os.chdir("..")

In [ ]:
if PSEUDO or VALIDATE:
    convertTrainLabel()

From some testing, this training takes about 5.5 hours to run. It doesn't depend on the amount of input images, only on the number of max batches, so it shouldn't be affected by the increase in test data. 

In [ ]:
if PSEUDO:
    makePseudolabel()

    if is_TEST:
        torch.cuda.empty_cache()
        generate_train()
        !mkdir backup # save in ../backup
        !rm obj.names
        !touch obj.names # data.names is the classes
        !echo "wheat" >> obj.names
        print('training')
        !./darknet detector train ../input/yolov4weights/trainsubmit.data ../input/yolov4weights/trainsubmit.cfg ../input/yolov4weights/submit.weights -dont_show -clear 1
        WEIGHTS = Darknet(cfgfile)
        WEIGHTS.load_weights('backup/trainsubmit_final.weights')
        WEIGHTS.cuda()
    else:
        pass

In [ ]:
if VALIDATE and is_TEST:
    all_predictions = validate()
    
    # Bayesian Optimization
    '''
    space = [
        Real(0, 1, name='iou_thr'),
        Real(0.25, 1, name='skip_box_thr'),
        Real(0, 1, name='score_threshold'),
    ]

    opt_result = optimize(
        space, 
        all_predictions,
        n_calls=50,
    )
    '''
    best_final_score = -opt_result.fun
    best_iou_thr = opt_result.x[0]
    best_skip_box_thr = opt_result.x[1]
    best_score_threshold = opt_result.x[2]


    print('-'*13 + 'WBF' + '-'*14)
    print("[Baseline score]", calculate_final_score(all_predictions, 0.6, 0.43, 0))
    print(f'[Best Iou Thr]: {best_iou_thr:.3f}')
    print(f'[Best Skip Box Thr]: {best_skip_box_thr:.3f}')
    print(f'[Best Score Thr]: {best_score_threshold:.3f}')
    print(f'[Best Score]: {best_final_score:.4f}')
    print('-'*30)
    
    
    for score_threshold in tqdm(np.arange(0, 1, 0.01), total=np.arange(0, 1, 0.01).shape[0]):
        final_score = calculate_final_score(all_predictions, best_iou_thr, best_skip_box_thr, score_threshold)
        if final_score > best_final_score:
            best_final_score = final_score
            best_score_threshold = score_threshold

    print('-'*30)
    print(f'[Best Score Threshold]: {best_score_threshold}')
    print(f'[OOF Score]: {best_final_score:.4f}')
    print('-'*30)

In [ ]:
def format_prediction_string(boxes, scores):
    pred_strings = []
    for j in zip(scores, boxes):
        pred_strings.append("{0:.4f} {1} {2} {3} {4}".format(j[0], j[1][0], j[1][1], j[1][2], j[1][3]))

    return " ".join(pred_strings)

def detect():
    source = '../input/global-wheat-detection/test/'
    weights = WEIGHTS
    
    imagenames =  os.listdir(source)
    
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    # Load model
    model = WEIGHTS  # load to FP32
    model.to(device).eval()
    
    dataset = LoadImages(source, img_size=1024)

    results = []
    fig, ax = plt.subplots(5, 2, figsize=(30, 70))
    count = 0
    
    for path, img, img0, vid_cap in dataset:
        image_id = os.path.basename(path).split('.')[0]
        img = img.transpose(1,2,0) # [H, W, 3]
        
        enboxes = []
        enscores = []
        
        # only rot, no flip
        if is_ROT:    
            for i in range(4):
                img1 = TTAImage(img, i)
                boxes, scores = detect1Image(img1, img0, model, device, aug=False)
                for _ in range(3-i):
                    boxes = rotBoxes90(boxes, *img.shape[:2])            
                enboxes.append(boxes)
                enscores.append(scores) 
        
        # flip
        boxes, scores = detect1Image(img, img0, model, device, aug=is_AUG)
        enboxes.append(boxes)
        enscores.append(scores) 
            
        boxes, scores, labels = run_wbf(enboxes, enscores, image_size=1024, iou_thr=best_iou_thr, skip_box_thr=best_skip_box_thr)    
        boxes = boxes.astype(np.int32).clip(min=0, max=1023)
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
        
        indices = scores >= best_score_threshold
        boxes = boxes[indices]
        scores = scores[indices]
        
        if count<10:
            img_ = cv2.imread(path)  # BGR
            img_ = cv2.cvtColor(img_, cv2.COLOR_BGR2RGB)
            for box, score in zip(boxes,scores):
                cv2.rectangle(img_, (box[0], box[1]), (box[2]+box[0], box[3]+box[1]), (220, 0, 0), 2)
                cv2.putText(img_, '%.2f'%(score), (box[0], box[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
            ax[count%5][count//5].imshow(img_)
            count+=1
            
        result = {
            'image_id': image_id,
            'PredictionString': format_prediction_string(boxes, scores)
        }

        results.append(result)
    return results

In [ ]:
results = detect()
test_df = pd.DataFrame(results, columns=['image_id', 'PredictionString'])
!rm -rf ./*
test_df.to_csv('submission.csv', index=False)
test_df.head()